In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install git+https://github.com/facebookresearch/fastText

  Cloning https://github.com/facebookresearch/fastText to /tmp/pip-req-build-ps2x7zt2
  Running command git clone -q https://github.com/facebookresearch/fastText /tmp/pip-req-build-ps2x7zt2
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015273 sha256=7118a6512ea35f55773b3a55b7c14f0ed0ce7067c104cbe0ce92c6fb53751d2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-kn07n4ec/wheels/72/f6/b9/3a859d37e24c73635e6c6af085ad99829e8b2ab1bd965b15a9
Successfully built fasttext


In [3]:
#%tensorflow_version 2.x

In [4]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [5]:
!ls /content/drive/My\ Drive/Code/data_aumentation_for_author_profiling

figures  notebooks  word_level_da


In [6]:
!ls /content/drive/My\ Drive/Data/depresion/2019/train

augmented		   original_prep
class_distpng		   original_prep_ground_truth.txt
golden_truth_filtered.txt  prep_chunks
golden_truth.txt	   prep_chunks_filtered_neg_only
ground_truth.txt	   train_golden_truth_joined.txt


In [7]:
import sys
sys.path.append('/content/drive/My Drive/Code/data_aumentation_for_author_profiling/')

In [8]:

from datetime import datetime
from word_level_da import utils
from word_level_da.preprocessing.load_data import Dataset

In [9]:
  epochs= 20      
  layers=1
  nodes=256
  dim=300
  label_pos=1
  len_doc=64
  key="depresion19_drive"
  LOAD_EMB=False
  fast_file='cc.en.300.bin'
  

In [10]:
if LOAD_EMB:
  import fasttext.util
  fasttext.util.download_model('en', if_exists='ignore')  # English
  ft = fasttext.load_model('cc.en.300.bin')

In [11]:
from word_level_da.classifier.train_sequence_model import seq_model
import numpy as np

In [12]:
data = Dataset(key=key, doc_len=len_doc, min_len=int(len_doc / 2), chunking=True, remove_end=True)

In [13]:
training, test=data.get_train_test(return_ids=True)

In [14]:
bi_gru=seq_model(weights_path="/content/drive/My Drive/Code/datadata_aumentation_for_author_profiling/weights/", 
                 static=False, load_all_vectors=LOAD_EMB,
                 ids_labels=dict.fromkeys(test[2]).keys(), original_labels=test[3][0])

In [15]:
n_docs=[i for i in range(1,11)]
ratio=0
augmentation_method="Over"
model="rnn-fixed"
umbral=0.5
score_m="avg"
q=75

In [16]:
fold=0
n=0
#ratio=0
r=0.2

In [17]:
scores=[]
AUGMENTED=False

In [18]:
!ls "/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj/"

predictions_depresion19_localrnn-fixed.csv  WE_depresion19.pkl


In [ ]:
for n in n_docs:
      prefix = augmentation_method + str(n)
      folder = augmentation_method + "/" + prefix
      truth_file = augmentation_method + "/" + prefix + ".txt"

      
      if AUGMENTED:
        docs, l_docs, ids, useless_data = data.get_dataset(folder_name=folder, truth_name=truth_file,
                                                               partition="augmented")
                
        new_training=np.append(training[0],docs)
        new_labels=np.append( training[1], l_docs)
      else:
        new_training=training[0]
        new_labels=training[1]

      info=bi_gru.buil_model(((new_training, new_labels), (test[0], test[1])), layers=1, nodes=nodes, embedding_dim=dim, dropout_rate=r, TOP_K=150000, 
                                pretrained=True, embedding_trainable=False, bidirectional=True, 
                          seq_len=len_doc, emb_file=fast_file, class_imbanlance=True, algo=model,
                          kernel_size=None, vocab_dir="/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj",
                           key=key.split("_")[0])
      info[2]=umbral
      for i in range(3):
        score=bi_gru.train_model(1e-3, epochs=epochs, patience=3, batch_size=1024, load_weights=False, save_weigths=False, 
                                                weights_name=key.split("_")[0]+prefix+model+str(i)+".h5",
                                  ad_data=(test[3]), monitor_measure="val_loss", validation=True,
                                 umbral=umbral,  score_method=score_m, q=q)
        
        final_info=[augmentation_method,n,i ]
        final_info+=info+list(score)
        scores.append(final_info)
        print(final_info)


Number of new embeddings:  0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 300)           35600700  
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               1140736   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 36,873,021
Trainable params: 1,272,321
Non-trainable params: 35,600,700
_______

In [ ]:
for score in scores:
  print(score)

In [ ]:
bi_gru.save_predictions("/content/drive/My Drive/Code/data_aumentation_for_author_profiling/predictions/", "predictions_"+model+"_"+method)    
logger.info("Finish Time: %s", datetime.now())